In [30]:
import requests
import mysql.connector
import re
from unidecode import unidecode

In [31]:
db_cnx = mysql.connector.connect(user="docker", password="docker", host="localhost", database="tcc")

In [32]:
dataInicio = "2022-01-01"
dataFim = "2022-12-01"

## Buscando proposições

In [33]:
listagem_proposicoes_url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes"
detalhe_proposicoes_url = "https://dadosabertos.camara.leg.br/api/v2/proposicoes/"
##TODO: salvar isso aqui num arquivo e só consultar (ou fazer um cache)
codigo_situacao_url = "https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/codSituacao"

params = {
    "ordenarPor": "siglaTipo",
    "dataApresentacaoInicio": dataInicio,
    "dataApresentacaoFim": dataFim,
    "siglaTipo": "PL,PLC",
    "itens": "10"
}
response = requests.get(listagem_proposicoes_url, params).json()

In [34]:
proposicoes = response["dados"]

In [44]:
codigo_situacao_lista = requests.get(codigo_situacao_url).json()["dados"]
dicionario_cod_situacao = {}
for i in codigo_situacao_lista:
    dicionario_cod_situacao[i["cod"]] = i["nome"]

In [36]:
query = """
insert into proposicoes(id, tipo, ano, ementa, ementa_detalhada, uri_documento, data_apresentacao, ultimo_relator_id,
                        situacao)
    value (%(id)s, %(siglaTipo)s, %(ano)s, %(ementa)s, null, %(uri)s, null, null, null)
on duplicate key update tipo=%(siglaTipo)s, ano=%(ano)s, ementa=%(ementa)s, ementa_detalhada=null, uri_documento=null,
                        data_apresentacao= null, ultimo_relator_id = null, situacao = null;
"""
## Campos em nulo serão atualizados mais tarde
cursor = db_cnx.cursor()
for preposicao in proposicoes:
    cursor.execute(query, preposicao)
db_cnx.commit()
cursor.close()

True

In [37]:
def extrair_id_deputado(url):
    return re.search(r"/deputados/(.*)$", url).group(1)

In [46]:
query = """
update proposicoes set
                       ementa_detalhada = %(ementaDetalhada)s, data_apresentacao = %(dataApresentacao)s,
                       uri_documento = %(urlInteiroTeor)s,
                       ultimo_relator_id = %(ultimoRelatorId)s, situacao = %(situacao)s where id = %(id)s ;
"""

query_keywords = """
    insert into proposicoes_keywords(keyword, proposicao_id) values (%(keyword)s, %(id)s);
"""
preposicao_updates = []
cursor = db_cnx.cursor()
for preposicao in proposicoes:
    response = requests.get(f"{detalhe_proposicoes_url}/{preposicao['id']}").json()["dados"]

    params = {
        "ementaDetalhada": response["ementaDetalhada"],
        "dataApresentacao": response["dataApresentacao"][:-6],
        "id": preposicao['id'],
        "urlInteiroTeor": response["urlInteiroTeor"]
    }
    uriUltimoRelator = response["statusProposicao"]["uriUltimoRelator"]
    ultimoRelatorId = None
    if not uriUltimoRelator is None:
        ultimoRelatorId = re.search(r"/deputados/(.*)$", response["statusProposicao"]["uriUltimoRelator"]).group(1)

    autores = requests.get(response["uriAutores"]).json()["dados"]

    for autor in autores:
        if autor.get("tipo") == "Deputado":
            id_autor = extrair_id_deputado(autor.get("uri"))

            cursor.execute("select exists(select * from deputado where id = %s)", [id_autor])
            deputado_existe = cursor.fetchone()[0] == 1

            if deputado_existe:
                cursor.execute("""
                    insert IGNORE into autor_proposicoes (id_proposicao, id_deputado)
                    values (%s, %s);
                """, (params["id"], id_autor))
    cod_situacao = response["statusProposicao"]["codSituacao"]
    params["situacao"] = None
    if not cod_situacao is None:
        params["situacao"] = dicionario_cod_situacao[str(cod_situacao)]
    params["ultimoRelatorId"] = ultimoRelatorId

    cursor.execute(query, params)

    cursor.execute("delete from proposicoes_keywords where proposicao_id = %(id)s", {"id": params["id"]})
    for i in response["keywords"].split(", "):
        cursor.execute(query_keywords, {"keyword": unidecode(i.strip().lower().replace(".", "")), "id": params["id"]})
db_cnx.commit()
cursor.close()

True